<a href="https://colab.research.google.com/github/myers1091/UmpqQuant_GroupLearningSeminar/blob/main/hnorem/session05/ch_13_exercise_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.train import Feature, Features, Example, BytesList, FloatList, Int64List
import matplotlib.pyplot as plt

import re
import shutil
import string

from tensorflow.keras import utils
import collections
import pathlib


# Exercise! 

Let's repeat what we did for exercise 10 of chapter 13, but on the dataset below. What needs to be adapted and how? I have no idea :D

I have included my solution below this.

In [22]:
data_url = 'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'

dataset_dir = utils.get_file(
    origin=data_url,
    untar=True,
    cache_dir='stack_overflow',
    cache_subdir='')

dataset_dir = pathlib.Path(dataset_dir).parent

6053168/6053168 [==============================] - 0s 0us/step


In [23]:
list(dataset_dir.iterdir())

[PosixPath('/tmp/.keras/test'),
 PosixPath('/tmp/.keras/stack_overflow_16k.tar.gz'),
 PosixPath('/tmp/.keras/README.md'),
 PosixPath('/tmp/.keras/train')]

In [24]:
train_dir = dataset_dir/'train'
list(train_dir.iterdir())

[PosixPath('/tmp/.keras/train/java'),
 PosixPath('/tmp/.keras/train/python'),
 PosixPath('/tmp/.keras/train/csharp'),
 PosixPath('/tmp/.keras/train/javascript')]

In [25]:
sample_file = train_dir/'python/1755.txt'

with open(sample_file) as f:
  print(f.read())

why does this blank program print true x=true.def stupid():.    x=false.stupid().print x



Cheat sheet available here but let's try to look as little as possible:

https://www.tensorflow.org/tutorials/load_data/text

## Exercise 10

Exercise: In this exercise you will download a dataset, split it, create a tf.data.Dataset to load it and preprocess it efficiently, then build and train a binary classification model containing an Embedding layer.

###PART A - C. 

Download the Large Movie Review Dataset, which contains 50,000 movies reviews from the Internet Movie Database. The data is organized in two directories, train and test, each containing a pos subdirectory with 12,500 positive reviews and a neg subdirectory with 12,500 negative reviews. Each review is stored in a separate text file. There are other files and folders (including preprocessed bag-of-words), but we will ignore them in this exercise.

Split validation into test and validation.

Use tf.data to create an efficient dataset for each set.

In [2]:
from pathlib import Path

DOWNLOAD_ROOT = "http://ai.stanford.edu/~amaas/data/sentiment/"
FILENAME = "aclImdb_v1.tar.gz"
filepath = keras.utils.get_file(FILENAME, DOWNLOAD_ROOT + FILENAME, extract=True)
path = Path(filepath).parent / "aclImdb"
path

84125825/84125825 [==============================] - 2s 0us/step


PosixPath('/root/.keras/datasets/aclImdb')

In [3]:
import os
for name, subdirs, files in os.walk(path):
    indent = len(Path(name).parts) - len(path.parts)
    print("    " * indent + Path(name).parts[-1] + os.sep)
    for index, filename in enumerate(sorted(files)):
        if index == 3:
            print("    " * (indent + 1) + "...")
            break
        print("    " * (indent + 1) + filename)

aclImdb/
    README
    imdb.vocab
    imdbEr.txt
    test/
        labeledBow.feat
        urls_neg.txt
        urls_pos.txt
        pos/
            0_10.txt
            10000_7.txt
            10001_9.txt
            ...
        neg/
            0_2.txt
            10000_4.txt
            10001_1.txt
            ...
    train/
        labeledBow.feat
        unsupBow.feat
        urls_neg.txt
        ...
        unsup/
            0_0.txt
            10000_0.txt
            10001_0.txt
            ...
        pos/
            0_9.txt
            10000_8.txt
            10001_10.txt
            ...
        neg/
            0_3.txt
            10000_4.txt
            10001_4.txt
            ...


In [4]:
def review_paths(dirpath):
    return [str(path) for path in dirpath.glob("*.txt")]

train_pos = review_paths(path / "train" / "pos")
train_neg = review_paths(path / "train" / "neg")
test_valid_pos = review_paths(path / "test" / "pos")
test_valid_neg = review_paths(path / "test" / "neg")

len(train_pos), len(train_neg), len(test_valid_pos), len(test_valid_neg)

(12500, 12500, 12500, 12500)

In [5]:
np.random.shuffle(test_valid_pos)

test_pos = test_valid_pos[:5000]
test_neg = test_valid_neg[:5000]
valid_pos = test_valid_pos[5000:]
valid_neg = test_valid_neg[5000:]

In [6]:
def imdb_dataset(filepaths_positive, filepaths_negative, n_read_threads=5):
  positive_reviews_ds = tf.data.TextLineDataset(filepaths_positive, num_parallel_reads=n_read_threads)
  positive_reviews_ds = positive_reviews_ds.map(lambda review: (review, 1))

  negative_reviews_ds = tf.data.TextLineDataset(filepaths_negative, num_parallel_reads=n_read_threads)
  negative_reviews_ds = negative_reviews_ds.map(lambda review: (review, 0))
  
  return tf.data.Dataset.concatenate(positive_reviews_ds, negative_reviews_ds)

In [7]:
example_ds = tf.data.TextLineDataset(train_pos[0:3], num_parallel_reads=5)

In [8]:
list(example_ds.as_numpy_iterator())

[b"CRY FREEDOM is an excellent primer for those wanting an overview of apartheid's cruelty in just a couple of hours. Famed director Richard Attenborough (GANDHI) is certainly no stranger to the genre, and the collaboration of the real-life Mr. and Mrs. Woods, the main white characters in their book and in this film, lends further authenticity to CRY FREEDOM. The video now in release actually runs a little over 2 and a half hours since 23 minutes of extra footage was inserted to make it a two part TV miniseries after the film's initial theatrical release. While the added length serves to heighten the film's forgivable flaws: uneven character development and blanket stereotyping in particular, another possible flaw (the insistence on the white characters' fate over that of the African ones) may work out as a strength. Viewing CRYING FREEDOM as a politically and historically educational film (as I think it should, over its artistic merits), the story is one which black Africans know only

In [9]:
train_ds = imdb_dataset(train_pos, train_neg)
valid_ds = imdb_dataset(valid_pos, valid_neg)
test_ds = imdb_dataset(test_pos, test_neg)

In [10]:
batch_size = 32

train_ds = train_ds.shuffle(25000).batch(batch_size).prefetch(1)
valid_ds = valid_ds.batch(batch_size).prefetch(1)
test_ds = test_ds.batch(batch_size).prefetch(1)

### PART D.

Create a binary classification model, using a TextVectorization layer to preprocess each review. If the TextVectorization layer is not yet available (or if you like a challenge), try to create your own custom preprocessing layer: you can use the functions in the tf.strings package, for example lower() to make everything lowercase, regex_replace() to replace punctuation with spaces, and split() to split words on spaces. You should use a lookup table to output word indices, which must be prepared in the adapt() method.

In [11]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  no_punctuation = tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')
  return no_punctuation

text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens = 1000,
    standardize = custom_standardization
)
  

In [12]:
# this was kind of slow, so we'll do the sample thing
#text_vectorizer.adapt(train_ds.map(lambda review, label: review))

sample_train_ds = train_ds.take(500)

text_vectorizer.adapt(sample_train_ds.map(lambda review, label: review))

In [13]:
#Had a heck of a time getting this to run!

class Bag_of_words_layer(keras.layers.Layer):
  def __init__(self, n_tokens, dtype=tf.int32, **kwargs):
    super().__init__(trainable = False, dtype=dtype, **kwargs)
    self.n_tokens = n_tokens

  def call(self, inputs):
    one_hot_encoded = tf.one_hot(inputs, self.n_tokens)
    return tf.reduce_sum(one_hot_encoded, axis=1)

In [14]:
model = tf.keras.Sequential([
    text_vectorizer,
    Bag_of_words_layer(len(text_vectorizer.get_vocabulary())),
    keras.layers.Dense(100),
    keras.layers.Dense(1, activation = "sigmoid")
]
)

model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])

In [15]:
model.fit(train_ds, epochs=5, validation_data=valid_ds)

Epoch 1/5
782/782 [==============================] - 89s 108ms/step - loss: 0.7327 - accuracy: 0.7594 - val_loss: 0.3824 - val_accuracy: 0.8543
Epoch 2/5
782/782 [==============================] - 74s 82ms/step - loss: 0.5725 - accuracy: 0.8181 - val_loss: 0.4686 - val_accuracy: 0.8387
Epoch 3/5
782/782 [==============================] - 70s 85ms/step - loss: 0.5055 - accuracy: 0.8254 - val_loss: 0.4151 - val_accuracy: 0.8415
Epoch 4/5
782/782 [==============================] - 64s 79ms/step - loss: 0.4687 - accuracy: 0.8300 - val_loss: 0.3765 - val_accuracy: 0.8537
Epoch 5/5
782/782 [==============================] - 67s 82ms/step - loss: 0.4427 - accuracy: 0.8353 - val_loss: 0.4284 - val_accuracy: 0.8322


### PART E.

Add an Embedding layer and compute the mean embedding for each review, multiplied by the square root of the number of words (see Chapter 16). This rescaled mean embedding can then be passed to the rest of your model.

Need to compute: number of words in review, excluding zero vectors; mean across words.


I don't understand the errors I am getting so I will recreate the pieces from scratch.

In [16]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  no_punctuation = tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')
  return no_punctuation

text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens = 1000,
    standardize = custom_standardization
)

In [17]:
sample_train_ds = train_ds.take(500)

text_vectorizer.adapt(sample_train_ds.map(lambda review, label: review))

In [18]:

embedding_dim = 16

embedding_layer = keras.layers.Embedding(input_dim = len(text_vectorizer.get_vocabulary()), 
                           output_dim = embedding_dim, mask_zero=True)

In [19]:


model = keras.Sequential([
    text_vectorizer,
    embedding_layer,
    keras.layers.Lambda(lambda x: tf.math.reduce_mean(x, axis=1)),
    keras.layers.Dense(100),
    keras.layers.Dense(1, activation = "sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])

model.fit(train_ds, epochs=5, validation_data=valid_ds)

Epoch 1/5
782/782 [==============================] - 24s 22ms/step - loss: 0.5457 - accuracy: 0.7150 - val_loss: 0.4435 - val_accuracy: 0.7789
Epoch 2/5
782/782 [==============================] - 23s 25ms/step - loss: 0.3691 - accuracy: 0.8499 - val_loss: 0.3618 - val_accuracy: 0.8509
Epoch 3/5
782/782 [==============================] - 20s 22ms/step - loss: 0.3441 - accuracy: 0.8596 - val_loss: 0.3406 - val_accuracy: 0.8607
Epoch 4/5
782/782 [==============================] - 20s 20ms/step - loss: 0.3320 - accuracy: 0.8644 - val_loss: 0.3371 - val_accuracy: 0.8621
Epoch 5/5
782/782 [==============================] - 17s 18ms/step - loss: 0.3274 - accuracy: 0.8681 - val_loss: 0.3359 - val_accuracy: 0.8622
